# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import nltk

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier

import pickle

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///MessagesDatabase.db')
df = pd.read_sql_table('Messages', con=engine)
# Set X: message column, Y: all comuns from column #4
X = df['message']
Y = df.iloc[: , 4:]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    '''
    Tokenize the text
    Params:
        text: string 
    '''
    text = text.lower()
    return word_tokenize(text)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# Create a pipieline with CountVectorizer, TfidfTransformer, and DecisionTreeClassifier
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
# train classifier
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# predict on test data
Y_pred = pipeline.predict(X_test)

In [8]:
def test_model(y_test, y_pred):    
    '''
    Test model using classification_report function and calculate Average accuracy
    Params:
        y_test: labels for test set
        y_pred: predicted labels
    '''
    for index, value in enumerate(y_test):
        print(value, classification_report(y_test[value], y_pred[:, index]))
    print('Average accuracy: ', (y_pred == Y_test).mean().mean())

In [9]:
test_model(Y_test, Y_pred)

related              precision    recall  f1-score   support

          0       0.44      0.43      0.44      1516
          1       0.83      0.83      0.83      4994
          2       0.33      0.20      0.25        44

avg / total       0.73      0.74      0.74      6554

request              precision    recall  f1-score   support

          0       0.91      0.92      0.91      5439
          1       0.58      0.57      0.58      1115

avg / total       0.86      0.86      0.86      6554

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      0.99      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.72      0.73      0.72      3826
          1       0.61      0.59      0.60      2728

avg / total       0.67      0.67      0.67      6554

medical_help              precision    recall  f1-sco

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {
        'vect__max_df': [10, 20],
        'vect__max_features': (5, 10),
        'clf__estimator__max_depth' : [20, 30, 50]
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

In [11]:
# training
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__max_df': [10, 20], 'vect__max_features': (5, 10), 'clf__estimator__max_depth': [20, 30, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
# predict model on X_test
Y_pred_improve = cv.predict(X_test)

In [13]:
# test model
test_model(Y_test, Y_pred_improve)

related              precision    recall  f1-score   support

          0       0.00      0.00      0.00      1516
          1       0.76      1.00      0.86      4994
          2       0.00      0.00      0.00        44

avg / total       0.58      0.76      0.66      6554

request              precision    recall  f1-score   support

          0       0.83      1.00      0.91      5439
          1       0.00      0.00      0.00      1115

avg / total       0.69      0.83      0.75      6554

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.58      1.00      0.74      3826
          1       0.40      0.00      0.00      2728

avg / total       0.51      0.58      0.43      6554

medical_help              precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
# show best parameters
cv.best_params_

{'clf__estimator__max_depth': 20, 'vect__max_df': 20, 'vect__max_features': 10}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
# Create a pipieline with CountVectorizer, TfidfTransformer, and RandomForestClassifier
pipeline_rf = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [16]:
# build a model using RandomForestClassifier algorithm
model = pipeline_rf.fit(X_train, Y_train)

In [17]:
# Predict the model with X_test set
Y_pred_rf = pipeline_rf.predict(X_test)
test_model(Y_test, Y_pred_rf)

related              precision    recall  f1-score   support

          0       0.61      0.34      0.44      1516
          1       0.82      0.93      0.87      4994
          2       0.80      0.09      0.16        44

avg / total       0.77      0.79      0.77      6554

request              precision    recall  f1-score   support

          0       0.88      0.99      0.93      5439
          1       0.85      0.37      0.52      1115

avg / total       0.88      0.88      0.86      6554

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.72      0.89      0.80      3826
          1       0.77      0.53      0.62      2728

avg / total       0.74      0.74      0.73      6554

medical_help              precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [18]:
filename = 'model.pkl'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.